# Coloring Algorithm
Final code / workflow for the coloring algorithm.

In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import sys

sys.path.append('../')
from algorithm.Voxel import Voxel
from algorithm.Lattice import Lattice
from algorithm.Surroundings import Surroundings
from algorithm.SymmetryDf import SymmetryDf
from algorithm.BondPainter import Mesovoxel, BondPainter
from algorithm.Rotation import VoxelRotater
from algorithm.ColorTree import ColorTree

from app.design.Designer import RunDesigner
from app.visualize.Visualizer import RunVisualizer

import numpy as np
import pandas as pd
from PyQt6.QtWidgets import QApplication
from PyQt6.QtCore import QCoreApplication

## The algorithm

### Create the desired lattice structure

Saves it to a file `data/lattice2.npy` to be loaded in a separate cell. Alternatively, you can define your own numpy array in code and create the lattice that way.

In [7]:
%gui qt

if __name__ == '__main__':
    if not QCoreApplication.instance():
        app = QApplication(sys.argv)
    else:
        app = QCoreApplication.instance()
    
    designer = RunDesigner(app)
    input_lattice = designer.run()
    
    if input_lattice is not None:
        print(f'Lattice received.')
        np.save("data/perovskite.npy", input_lattice)
    else:
        print("No lattice received.")

No lattice received.


### Run the algorithm

1. Creates lattice structure with voxel, bond objects.
2. Create the full surroundings matrix which is used for comparing symmetry operations on a tiled lattice structure.
3. Paint the bond with the algorithm

In [71]:
input_lattice = np.load('data/double_oreo.npy')

lattice = Lattice(input_lattice)
lattice.compute_symmetries()

# Bond painter stuff
bond_painter = BondPainter(lattice)
bond_painter.paint_lattice()

Initializing the mesovoxel...
Found best rotation between voxel0 and voxel3: translation
Found best rotation between voxel0 and voxel13: 180° X-axis
Found best rotation between voxel0 and voxel14: 180° X-axis
Found best rotation between voxel0 and voxel15: 180° X-axis
Found best rotation between voxel4 and voxel7: translation
Found best rotation between voxel4 and voxel9: 180° X-axis
Found best rotation between voxel4 and voxel10: 180° X-axis
Found best rotation between voxel4 and voxel11: 180° X-axis
Coloring the lattice...
Done!


In [97]:
from algorithm.BindingFlexibility import BindingFlexibility
# print(lattice.get_voxel(0).color_dict())

bf = BindingFlexibility(lattice)
print(bf.get_sympartners(0))

bf1_lattice = bf.binding_flexibility_1()

{2: ['+x', '-x', '+y', '-y'], 3: ['+z'], -1: ['-z']}
[['+y', '-x', '+z', '-y', '+x']]


## Minimum origami
Once a valid coloring scheme is found, we find the complementary combination which produces the minimum unique number of origami.

In [72]:
# Compute the minimum number of origami via game tree search
# all_color_configs = lattice.init_all_color_configs()
# for color in all_color_configs:
#     print(f'Color {color} has {len(all_color_configs[color])} configurations.')

color_tree = ColorTree(lattice)
lattice.reset_color_config()
print("Initial unique origami:", lattice.unique_origami())
# reduced_color_configs = color_tree._reduce_color_configs(color_tree.all_color_configs)

optimal_path = color_tree.optimize(end_early=False)
print("Optimal path of color configurations:", optimal_path)

# # # Apply the optimal color_tree path to the lattice
# lattice.apply_color_configs(optimal_path)
print(f'Final unique origami: {lattice.unique_origami()}')

# color_tree.print_optimal_path(optimal_path)

Initial unique origami: [0, 1, 4, 5, 8, 9, 12, 13]
Evaluating color-wise configurations for 5 colors...
Evaluating color 5/5, config 16/16.....
Done with color-wise evaluation.
Color 1 has 16 configurations.
Color 2 has 4 configurations.
Color 3 has 16 configurations.
Color 4 has 4 configurations.
Color 5 has 16 configurations.
Searching for minimum origami across 65536 possibilities...
Evaluating color-wise configurations for 5 colors...
Evaluating color 5/5, config 16/16.....
Done with color-wise evaluation.
Evaluating color-wise configurations for 5 colors...
Evaluating color 5/5, config 2/2.....
Done with color-wise evaluation.
Found 6 new minimum unique origami. Reducing search space to 1024 possibilities...
Color 1 has 4 configurations.
Color 2 has 4 configurations.
Color 3 has 16 configurations.
Color 4 has 2 configurations.
Color 5 has 2 configurations.
Searching for minimum origami across 1024 possibilities...
Evaluating color-wise configurations for 5 colors...
Evaluating col

In [ ]:
for color, configs in reduced_color_configs.items():
    print(f'Color {color} has {len(configs)} configurations.')
    for config in configs:
        lattice.reset_color_config()
        lattice.apply_color_configs({color: config})
        unique_origami = lattice.unique_origami()
        print(f'{len(unique_origami)} Unique origami: {unique_origami}')

In [65]:
for color, voxels in lattice.init_colordict().items():
    print(f'Color {color}: {voxels}')

Color 1: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Color 2: [0, 1, 2, 3, 12, 13, 14, 15]
Color 3: [0, 1, 2, 3, 12, 13, 14, 15]
Color 4: [4, 5, 6, 7, 8, 9, 10, 11]
Color 5: [4, 5, 6, 7, 8, 9, 10, 11]


In [29]:
# Lists to store combined voxel and bond information
final_df = lattice.final_df(show_bond_type=False)
final_df

Voxel                      Bond Colors               
      ID Material Coordinates          +x -x +y -y +z -z
0      0        1   (0, 1, 3)           2  2  2  2  3  1
1      1        1   (1, 1, 3)          -2 -2 -2 -2  3  1
2      2        1   (0, 0, 3)          -2 -2 -2 -2  3  1
3      3        1   (1, 0, 3)           2  2  2  2  3  1
4      4        0   (0, 1, 2)           4  4  4  4 -1  5
5      5        0   (1, 1, 2)          -4 -4 -4 -4 -1  5
6      6        0   (0, 0, 2)          -4 -4 -4 -4 -1  5
7      7        0   (1, 0, 2)           4  4  4  4 -1  5
8      8        0   (0, 1, 1)           4  4  4  4 -5 -1
9      9        0   (1, 1, 1)          -4 -4 -4 -4 -5 -1
10    10        0   (0, 0, 1)          -4 -4 -4 -4 -5 -1
11    11        0   (1, 0, 1)           4  4  4  4 -5 -1
12    12        1   (0, 1, 0)           2  2  2  2  1 -3
13    13        1   (1, 1, 0)          -2 -2 -2 -2  1 -3
14    14        1   (0, 0, 0)          -2 -2 -2 -2  1 -3
15    15        1   (1, 0, 0)           2  2  2  2  1 -3

In [30]:
final_df = lattice.final_df(show_bond_type=True)
final_df

Voxel                         Bond Colors                                \
      ID Material Coordinates             +x             -x             +y   
0      0        1   (0, 1, 3)  complementary  complementary  complementary   
1      1        1   (1, 1, 3)  complementary  complementary  complementary   
2      2        1   (0, 0, 3)  complementary  complementary  complementary   
3      3        1   (1, 0, 3)  complementary  complementary  complementary   
4      4        0   (0, 1, 2)  complementary  complementary  complementary   
5      5        0   (1, 1, 2)  complementary  complementary  complementary   
6      6        0   (0, 0, 2)  complementary  complementary  complementary   
7      7        0   (1, 0, 2)  complementary  complementary  complementary   
8      8        0   (0, 1, 1)  complementary  complementary  complementary   
9      9        0   (1, 1, 1)  complementary  complementary  complementary   
10    10        0   (0, 0, 1)  complementary  complementary  complementary   
11    11        0   (1, 0, 1)  complementary  complementary  complementary   
12    12        1   (0, 1, 0)  complementary  complementary  complementary   
13    13        1   (1, 1, 0)  complementary  complementary  complementary   
14    14        1   (0, 0, 0)  complementary  complementary  complementary   
15    15        1   (1, 0, 0)  complementary  complementary  complementary   

                                                 
               -y             +z             -z  
0   complementary  complementary     structural  
1   complementary  complementary     structural  
2   complementary  complementary     structural  
3   complementary  complementary     structural  
4   complementary     structural  complementary  
5   complementary     structural  complementary  
6   complementary     structural  complementary  
7   complementary     structural  complementary  
8   complementary  complementary     structural  
9   complementary  complementary     structural  
10  complementary  complementary     structural  
11  complementary  complementary     structural  
12  complementary     structural  complementary  
13  complementary     structural  complementary  
14  complementary     structural  complementary  
15  complementary     structural  complementary

### View the final colored lattice

In [99]:
%gui qt

if __name__ == '__main__':
    if not QCoreApplication.instance():
        app = QApplication(sys.argv)
    else:
        app = QCoreApplication.instance()

    # color_configs = reduced_color_configs[1]
    
    # lattice.reset_color_config()
    # lattice.apply_color_configs({1: reduced_color_configs[1][1]})
    # print(f"Unique origami: {lattice.unique_origami()}")
    visualizeWindow = RunVisualizer(bf1_lattice, app)

In [81]:
print(len(lattice.unique_origami()))
print(bond_painter.mesovoxel.structural_voxels)

4
{0, 4}


In [10]:
vox1_vox2_symlist = lattice.SymmetryDf.symlist(1, 2)
vox1_vox2_symlist

['90° Z-axis',
 '270° Z-axis',
 '180° X-axis + 270° Z-axis',
 '180° X-axis + 90° Z-axis',
 '270° Y-axis + 270° Z-axis',
 '270° Y-axis + 90° Z-axis',
 '270° Z-axis + 180° Y-axis',
 '90° X-axis + 270° Y-axis',
 '90° Y-axis + 270° Z-axis',
 '90° Y-axis + 90° Z-axis',
 '90° Z-axis + 180° Y-axis']

In [18]:
df = lattice.SymmetryDf.symmetry_df
df

,translation,90° X-axis,180° X-axis,270° X-axis,90° Y-axis,180° Y-axis,270° Y-axis,90° Z-axis,180° Z-axis,270° Z-axis,...,90° Y-axis + 180° X-axis,90° Y-axis + 180° Z-axis,90° Y-axis + 270° X-axis,90° Y-axis + 90° X-axis,90° Z-axis + 180° X-axis,90° Z-axis + 180° Y-axis,90° Z-axis + 270° X-axis,90° Z-axis + 270° Y-axis,90° Z-axis + 90° X-axis,90° Z-axis + 90° Y-axis
"(1, 4)",False,True,False,True,False,False,False,False,False,False,...,False,False,True,True,False,False,False,True,False,True
"(4, 6)",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
"(2, 3)",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
"(2, 6)",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
"(4, 5)",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
"(0, 5)",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
(1),True,False,True,False,True,True,True,False,True,False,...,True,True,False,False,False,False,False,False,False,False
"(3, 4)",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
"(0, 6)",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
"(1, 7)",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [72]:
lattice.SymmetryDf.symlist(1, 1)

['translation',
 '180° X-axis',
 '90° Y-axis',
 '180° Y-axis',
 '270° Y-axis',
 '180° Z-axis',
 '180° Y-axis + 180° X-axis',
 '180° Z-axis + 180° X-axis',
 '180° Z-axis + 180° Y-axis',
 '180° Z-axis + 90° Y-axis',
 '270° Y-axis + 180° X-axis',
 '270° Y-axis + 180° Z-axis',
 '90° Y-axis + 180° X-axis']

## Flip Complementarity Tests
We can also manually flip the complementarity. The following code produces the minimum result when applied to the perovskite.

In [69]:
voxel = lattice.get_voxel(8)
print(voxel.flip_complementarity(color=3))

voxel = lattice.get_voxel(4)
print(voxel.flip_complementarity(color=4))

voxel = lattice.get_voxel(7)
print(voxel.flip_complementarity(color=4))

# voxel.print_bonds()

[<algorithm.Voxel.Voxel object at 0x16d905340>, <algorithm.Voxel.Voxel object at 0x16d907cb0>, <algorithm.Voxel.Voxel object at 0x16d8d0b00>, <algorithm.Voxel.Voxel object at 0x16d8d1910>]
[<algorithm.Voxel.Voxel object at 0x15f3d1610>, <algorithm.Voxel.Voxel object at 0x15f3d2510>]
[<algorithm.Voxel.Voxel object at 0x16d823a40>, <algorithm.Voxel.Voxel object at 0x2889c4350>]


## Test post-processing speed

Found that deepcopy() created much of the time issues.

In [47]:
import cProfile
import pstats
import io
from pstats import SortKey

def profile_function(func, *args, **kwargs):
    pr = cProfile.Profile()
    pr.enable()
    result = func(*args, **kwargs)  # Run the function you want to profile
    pr.disable()

    s = io.StringIO()
    ps = pstats.Stats(pr, stream=s).sort_stats(SortKey.CUMULATIVE)
    ps.print_stats()
    print(s.getvalue())
    
    return result  # Return the result of the function

# Profile lattice.unique_origami()
profile_function(lattice.unique_origami)

# Profile lattice.apply_color_configs(optimal_path)
# profile_function(lattice.apply_color_configs, optimal_path)

         24441 function calls (24200 primitive calls) in 0.020 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.017    0.017 /Users/sarah/Desktop/ganglab/origami/origami-venv/lib/python3.12/site-packages/decorator.py:229(fun)
        1    0.000    0.000    0.017    0.017 /Users/sarah/Desktop/ganglab/origami/origami-venv/lib/python3.12/site-packages/IPython/core/history.py:55(only_when_enabled)
      2/1    0.000    0.000    0.017    0.017 /Users/sarah/Desktop/ganglab/origami/notebooks/../algorithm/Lattice.py:136(unique_origami)
       60    0.001    0.000    0.011    0.000 /Users/sarah/Desktop/ganglab/origami/notebooks/../algorithm/Rotation.py:178(rotate_voxel_bonds)
       30    0.000    0.000    0.008    0.000 /Users/sarah/Desktop/ganglab/origami/notebooks/../algorithm/SymmetryDf.py:50(symlist)
      168    0.000    0.000    0.005    0.000 /Users/sarah/Desktop/ganglab/origami/notebooks/..

[0, 1, 4, 5]